In [9]:
import pandas as pd
import numpy as np
import re
import json
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, roc_curve, precision_score, recall_score
import scikitplot as skplt
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [10]:
f = open('data/tweet_data/tweets_#superbowl.txt', "r")
wa_pat = re.compile('\\bWashington\\b|\\bWA\\b')
ma_pat = re.compile('\\bMassachusetts\\b|\\bMA\\b')

wa_tweets = []
ma_tweets = []

i = 0
for line in f:
    json_data = json.loads(line)
    if wa_pat.search(json_data['tweet']['user']['location']) and json_data['tweet']['lang'] == 'en':
        text = json_data['tweet']['text']
        location = json_data['tweet']['user']['location']
        time = json_data['citation_date']
        wa_tweets.append({'text':text, 'location':location, 'time':time})
    elif ma_pat.search(json_data['tweet']['user']['location'])  and json_data['tweet']['lang'] == 'en':
        text = json_data['tweet']['text']
        location = json_data['tweet']['user']['location']
        time = json_data['citation_date']
        ma_tweets.append({'text':text, 'location':location, 'time':time})


In [11]:
print('Washington', len(wa_tweets))
print('Masachussets', len(ma_tweets))

('Washington', 22581)
('Masachussets', 14386)


In [14]:
with open("data/washington.json",'w') as outfile:
    outfile.write(unicode(json.dumps(wa_tweets)))
with open("data/dmasachusetts.json",'w') as outfile:
    outfile.write(unicode(json.dumps(ma_tweets)))

In [15]:
wa_df = pd.DataFrame(wa_tweets)
ma_df = pd.DataFrame(ma_tweets)

wa_df = wa_df[['text','location', 'time']]
ma_df = ma_df[['text','location', 'time']]

In [16]:
ma_df.head()

,text,location,time
0,Our @ButchStearns talked #Patriots w/3X #Super...,"Boston, MA",1421496104
1,@jc_nogales Sunday February 1st. Se juega en A...,"Boston, MA, USA",1421238075
2,@Deadspin Is it worse than #NFL with #Patriots...,Central Massachusetts,1421240049
3,"@katyperry You have to play your ""Lose Your Lo...",Massachusetts,1421248501
4,@JessicCarpenter thanks for the follow Go #Pat...,Massachusetts,1421253334


In [25]:
ma_df_sorted = ma_df.sort_values(['time'])
wa_df_sorted = wa_df.sort_values(['time'])

In [24]:
ma_df_sorted.head()

,text,location,time
1,@jc_nogales Sunday February 1st. Se juega en A...,"Boston, MA, USA",1421238075
2,@Deadspin Is it worse than #NFL with #Patriots...,Central Massachusetts,1421240049
3,"@katyperry You have to play your ""Lose Your Lo...",Massachusetts,1421248501
4,@JessicCarpenter thanks for the follow Go #Pat...,Massachusetts,1421253334
5,@spirocks thanks for the follow Go #Patriots #...,Massachusetts,1421253430


In [26]:
wa_data = wa_df_sorted.as_matrix()

In [27]:
ma_data = ma_df_sorted.as_matrix()

In [32]:
wa_sent = []
ma_sent = []

In [33]:
print datetime.datetime.now().time()
for i in range(0,len(ma_data)):
    tweet = ma_data[i,0].encode('utf-8')
    sentiment = classifier.classify(tweet)
    ma_sent.append(sentiment)
    #print "%s: Classified tweet as %s." % (datetime.datetime.now().time(), sentiment)
print datetime.datetime.now().time()

17:54:43.755970


tweetment.py:135: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if word in w2c:
tweetment.py:203: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if token in lexicons[lexicon][sentiment]:
tweetment.py:228: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if token in lexicons[lexicon][sentiment]:


18:00:32.688459


In [37]:
print datetime.datetime.now().time()
for i in range(0,len(wa_data)):
    tweet = wa_data[i,0].encode('utf-8')
    sentiment = classifier.classify(tweet)
    wa_sent.append(sentiment)
    #print "%s: Classified tweet as %s." % (datetime.datetime.now().time(), sentiment)
print datetime.datetime.now().time()

18:03:18.321546
18:12:27.373937


In [39]:
ffile = open('data/ma_sent.txt', 'w')
for i in range(0,len(ma_sent)):
    ffile.write(ma_sent[i]+"\n")
ffile.close()

In [19]:
thefile = open('data/test1.txt', 'w')
#for i in range(0,len(wa_data)):
for i in range(0,50):
    thefile.write((wa_data[i,0].replace('\n', ' ')).encode('utf-8')+"\n")
thefile.close()

In [93]:
thefile = open('data/test_ma.txt', 'w')
for i in range(0,len(ma_data)):
#for i in range(0,10):
    thefile.write((ma_data[i,0].replace('\n', ' ')).encode('utf-8')+"\n")
thefile.close()

In [43]:
import datetime, time
import pytz
pst_tz = pytz.timezone('US/Pacific')

In [68]:
wa_dict = {}
ma_dict = {}

In [69]:
for i in range(0,len(wa_data)):
    d = datetime.datetime.fromtimestamp(wa_data[i,2], pst_tz)
    key = str(d.month) + "-" + str(d.day)
    if key in wa_dict:
        if wa_sent[i] == 'neutral':
            wa_dict[key][0]+=1
        elif wa_sent[i] == 'positive':
            wa_dict[key][1]+=1
        elif wa_sent[i] == 'negative':
            wa_dict[key][2]+=1
    else:
        if wa_sent[i] == 'neutral':
            wa_dict[key] = [1,0,0]
        elif wa_sent[i] == 'positive':
            wa_dict[key] = [0,1,0]
        elif wa_sent[i] == 'negative':
            wa_dict[key] = [0,0,1]

In [70]:
for i in range(0,len(ma_data)):
    d = datetime.datetime.fromtimestamp(ma_data[i,2], pst_tz)
    key = str(d.month) + "-" + str(d.day)
    if key in ma_dict:
        if ma_sent[i] == 'neutral':
            ma_dict[key][0]+=1
        elif ma_sent[i] == 'positive':
            ma_dict[key][1]+=1
        elif ma_sent[i] == 'negative':
            ma_dict[key][2]+=1
    else:
        if ma_sent[i] == 'neutral':
            ma_dict[key] = [1,0,0]
        elif ma_sent[i] == 'positive':
            ma_dict[key] = [0,1,0]
        elif ma_sent[i] == 'negative':
            ma_dict[key] = [0,0,1]

In [76]:
date_key =  ma_dict.keys()
date_key.sort()
date_key

['1-14',
 '1-15',
 '1-16',
 '1-17',
 '1-18',
 '1-19',
 '1-20',
 '1-21',
 '1-22',
 '1-23',
 '1-24',
 '1-25',
 '1-26',
 '1-27',
 '1-28',
 '1-29',
 '1-30',
 '1-31',
 '2-1',
 '2-2',
 '2-3',
 '2-4',
 '2-5',
 '2-6',
 '2-7']

In [95]:
for i in date_key:
    if i in ma_dict:
        ma_neutral.append(ma_dict[i][0])
        ma_positive.append(ma_dict[i][1])
        ma_negative.append(ma_dict[i][2])
    if i in wa_dict:
        wa_neutral.append(wa_dict[i][0])
        wa_positive.append(wa_dict[i][1])
        wa_negative.append(wa_dict[i][2])

In [118]:
def sumzip(*items):
    return [sum(values) for values in zip(*items)]

In [132]:
ma_neutral_norm = []
ma_positive_norm = []
ma_negative_norm = []

wa_neutral_norm = []
wa_positive_norm = []
wa_negative_norm = []

In [134]:
for i in date_key:
    if i in ma_dict:
        nsum = (ma_dict[i][0] + ma_dict[i][1] + ma_dict[i][2])*(1.0/100)
        ma_neutral_norm.append(ma_dict[i][0]/nsum)
        ma_positive_norm.append(ma_dict[i][1]/nsum)
        ma_negative_norm.append(ma_dict[i][2]/nsum)
    if i in wa_dict:
        nsum = (wa_dict[i][0] + wa_dict[i][1] + wa_dict[i][2])*(1.0/100)
        wa_neutral_norm.append(wa_dict[i][0]/nsum)
        wa_positive_norm.append(wa_dict[i][1]/nsum)
        wa_negative_norm.append(wa_dict[i][2]/nsum)

In [160]:
wa_dict_hourly = {}
ma_dict_hourly = {}

In [162]:
for i in range(0,len(wa_data)):
    d = datetime.datetime.fromtimestamp(wa_data[i,2], pst_tz)
    date = str(d.month) + "-" + str(d.day)
    key = d.hour
    if date == "2-1" and key in wa_dict_hourly:
        if wa_sent[i] == 'neutral':
            wa_dict_hourly[key][0]+=1
        elif wa_sent[i] == 'positive':
            wa_dict_hourly[key][1]+=1
        elif wa_sent[i] == 'negative':
            wa_dict_hourly[key][2]+=1
    elif date == "2-1":
        if wa_sent[i] == 'neutral':
            wa_dict_hourly[key] = [1,0,0]
        elif wa_sent[i] == 'positive':
            wa_dict_hourly[key] = [0,1,0]
        elif wa_sent[i] == 'negative':
            wa_dict_hourly[key] = [0,0,1]

In [164]:
for i in range(0,len(ma_data)):
    d = datetime.datetime.fromtimestamp(ma_data[i,2], pst_tz)
    date = str(d.month) + "-" + str(d.day)
    key = d.hour
    if date == "2-1" and key in ma_dict_hourly:
        if ma_sent[i] == 'neutral':
            ma_dict_hourly[key][0]+=1
        elif ma_sent[i] == 'positive':
            ma_dict_hourly[key][1]+=1
        elif ma_sent[i] == 'negative':
            ma_dict_hourly[key][2]+=1
    elif date == "2-1":
        if ma_sent[i] == 'neutral':
            ma_dict_hourly[key] = [1,0,0]
        elif ma_sent[i] == 'positive':
            ma_dict_hourly[key] = [0,1,0]
        elif ma_sent[i] == 'negative':
            ma_dict_hourly[key] = [0,0,1]

In [166]:
ma_neutral_hr = []
ma_positive_hr = []
ma_negative_hr = []

wa_neutral_hr = []
wa_positive_hr = []
wa_negative_hr = []

In [167]:
hour_key =  ma_dict_hourly.keys()
hour_key.sort()

In [168]:
for i in hour_key:
    if i in ma_dict_hourly:
        ma_neutral_hr.append(ma_dict_hourly[i][0])
        ma_positive_hr.append(ma_dict_hourly[i][1])
        ma_negative_hr.append(ma_dict_hourly[i][2])
    if i in wa_dict_hourly:
        wa_neutral_hr.append(wa_dict_hourly[i][0])
        wa_positive_hr.append(wa_dict_hourly[i][1])
        wa_negative_hr.append(wa_dict_hourly[i][2])

In [186]:
ma_neutral_norm_hr = []
ma_positive_norm_hr = []
ma_negative_norm_hr = []

wa_neutral_norm_hr = []
wa_positive_norm_hr = []
wa_negative_norm_hr = []

In [187]:
for i in hour_key:
    if i in ma_dict_hourly:
        nsum = (ma_dict_hourly[i][0] + ma_dict_hourly[i][1] + ma_dict_hourly[i][2])*(1.0/100)
        ma_neutral_norm_hr.append(ma_dict_hourly[i][0]/nsum)
        ma_positive_norm_hr.append(ma_dict_hourly[i][1]/nsum)
        ma_negative_norm_hr.append(ma_dict_hourly[i][2]/nsum)
    if i in wa_dict_hourly:
        nsum = (wa_dict_hourly[i][0] + wa_dict_hourly[i][1] + wa_dict_hourly[i][2])*(1.0/100)
        wa_neutral_norm_hr.append(wa_dict_hourly[i][0]/nsum)
        wa_positive_norm_hr.append(wa_dict_hourly[i][1]/nsum)
        wa_negative_norm_hr.append(wa_dict_hourly[i][2]/nsum)